<a href="https://colab.research.google.com/github/poojavjpy/Locust-Detection/blob/main/augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import random

def resize_to_same_dimensions(images, target_width=600, target_height=600):
    """Resize all images to the same width and height."""
    resized_images = []
    for image in images:
        resized_image = cv2.resize(image, (target_width, target_height))
        resized_images.append(resized_image)
    return resized_images

def add_border(image, border_size=10, color=(255, 255, 255)):
    """Add a border around the image."""
    return cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=color)

def put_title_on_image(image, title, font_scale=1.5, color=(255, 0, 0), thickness=3):
    """Place title text on the image at an appropriate location."""
    text_size = cv2.getTextSize(title, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]
    text_x = (image.shape[1] - text_size[0]) // 2
    text_y = text_size[1] + 10  # Add some padding from the top
    image = cv2.putText(image, title, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)
    return image

def combine_images(images, titles, output_path, max_images_per_row=3):
    """Combine images into a single large image with titles and save it."""
    images = resize_to_same_dimensions(images)
    images_with_borders = [add_border(image) for image in images]
    images_with_titles = [put_title_on_image(image, title) for image, title in zip(images_with_borders, titles)]
    num_rows = (len(images_with_titles) + max_images_per_row - 1) // max_images_per_row
    rows = []
    for i in range(num_rows):
        start_idx = i * max_images_per_row
        end_idx = min((i + 1) * max_images_per_row, len(images_with_titles))
        row_images = images_with_titles[start_idx:end_idx]
        rows.append(np.hstack(row_images))
    combined_image = np.vstack(rows)
    cv2.imwrite(output_path, cv2.cvtColor(combined_image, cv2.COLOR_RGB2BGR))
    print(f"All images combined and saved at {output_path}")

# Directory containing input images
input_folder = r"/content/drive/MyDrive/locust_detection/dataset/images/train"  # Update this path
output_dir = r"/content/drive/MyDrive/locust_detection/dataset/Augmented_Images/images/train"  # Update this path
os.makedirs(output_dir, exist_ok=True)

# Process each image in the folder
for filename in os.listdir(input_folder):
    input_image_path = os.path.join(input_folder, filename)
    if not os.path.isfile(input_image_path):
        continue  # Skip directories
    try:
        original_image = cv2.imread(input_image_path)
        if original_image is None:
            raise FileNotFoundError(f"Image not found at {input_image_path}")
        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        locust_bboxes = [(50, 50, 250, 250)]  # Placeholder for locust detection
        locust_bbox = locust_bboxes[0]
        cropped_locust = original_image_rgb[locust_bbox[1]:locust_bbox[3], locust_bbox[0]:locust_bbox[2]]
        flipped_image = cv2.flip(original_image_rgb, 1)
        vertical_flipped_image = cv2.flip(original_image_rgb, 0)
        brightness_image = cv2.convertScaleAbs(original_image_rgb, alpha=random.uniform(0.8, 1.2), beta=random.randint(-30, 30))
        rotated_scaled_image = cv2.warpAffine(original_image_rgb, cv2.getRotationMatrix2D((original_image_rgb.shape[1] / 2, original_image_rgb.shape[0] / 2), random.uniform(-15, 15), random.uniform(0.9, 1.1)), (original_image_rgb.shape[1], original_image_rgb.shape[0]))
        blurred_image = cv2.GaussianBlur(original_image_rgb, (random.choice([3, 5]), random.choice([3, 5])), 0)
        hsv_image = cv2.cvtColor(original_image_rgb, cv2.COLOR_RGB2HSV)
        hsv_image = cv2.merge([cv2.add(hsv_image[:, :, 0], random.randint(-10, 10)), hsv_image[:, :, 1], hsv_image[:, :, 2]])
        color_shifted_image = cv2.merge([cv2.add(original_image_rgb[:, :, 0], random.randint(-20, 20)), cv2.add(original_image_rgb[:, :, 1], random.randint(-20, 20)), cv2.add(original_image_rgb[:, :, 2], random.randint(-20, 20))])
        augmented_images = [original_image_rgb, flipped_image, vertical_flipped_image, brightness_image, rotated_scaled_image, blurred_image, hsv_image, color_shifted_image, cropped_locust]
        titles = ['Original', 'Flipped', 'Vertical Flip', 'Brightness', 'Rotation/Scale', 'Gaussian Blur', 'HSV Adjust', 'Color Shift', 'Cropped']
        output_path = os.path.join(output_dir, f"combined_{filename}")
        combine_images(augmented_images, titles, output_path)
    except Exception as e:
        print(f"Error processing {filename}: {e}")